In [18]:
import numpy as np
import gym
from gym import spaces
import random
import pandas as pd

**action**:
* Đánh 1 cây cùng chất
* Đánh 2
* Đánh 1 đôi
* Đánh 1 bộ ba
* Đánh tứ quý

* Đánh dây 3
* Đánh dây 4
* Đánh dây 5
* Đánh dây 6
* Đánh dây 7
* Đánh dây 8
* Đánh dây 9
* Đánh dây 10
* Đánh dây 11
* Đánh dây 12
* Đánh dây 13

* Bỏ lượt

In [17]:
data_cards = ['3_bich', '3_nhep', '3_do', '3_co' , '4_bich', '4_nhep', '4_do', '4_co', '5_bich', '5_nhep', '5_do', '5_co', '6_bich', '6_nhep', '6_do', '6_co',  '7_bich', '7_nhep', '7_do', '7_co',  '8_bich', '8_nhep', '8_do', '8_co', '9_bich', '9_nhep', '9_do', '9_co', '10_bich', '10_nhep', '10_do', '10_co', 'j_bich', 'j_nhep', 'j_do', 'j_co',  'q_bich', 'q_nhep', 'q_do', 'q_co', 'k_bich', 'k_nhep', 'k_do', 'k_co',  'A_bich', 'A_nhep', 'A_do', 'A_co', '2_bich', '2_nhep', '2_do', '2_co']

In [139]:
data_actions = [
            'play_single',
            'play_two',
            'play_pair',
            'play_trio',
            'play_four_of_a_kind',
            'play_straight_3',
            'play_straight_4',
            'play_straight_5',
            'play_straight_6',
            'play_straight_7',
            'play_straight_8',
            'play_straight_9',
            'play_straight_10',
            'play_straight_11',
            'play_straight_12',
            'play_straight_13',
            'pass_turn']

In [140]:
cards_data = pd.DataFrame(data_cards)
action_data = pd.DataFrame(data_actions)
action_data

,0
0,play_single
1,play_two
2,play_pair
3,play_trio
4,play_four_of_a_kind
5,play_straight_3
6,play_straight_4
7,play_straight_5
8,play_straight_6
9,play_straight_7


* mod 4 = 0 => bích
* mod 4 = 1 => nhép
* mod 4 = 2 => dô
* mod 4 = 3 => cơ


In [31]:
class TLMB_Env(gym.Env):
    def __init__(self):
        super(TLMB_Env, self).__init__()
        self.num_cards = 52
        self.num_players = 4
        self.num_cards_per_player = 13
        
        self.action_space = spaces.Discrete(17)    #num_actions = 17
        self.observation_space = spaces.Dict({
            'player_hand': spaces.MultiBinary(self.num_cards),     
            'opponent_hand': spaces.MultiBinary((self.num_players - 1, self.num_cards)),
            'played_cards': spaces.MultiBinary(self.num_cards),
            'current_played_cards': spaces.MultiBinary(self.num_cards)
        })
        self.actions = [
            'play_single',
            'play_two',
            'play_pair',
            'play_trio',
            'play_four_of_a_kind',
            'play_straight_3',
            'play_straight_4',
            'play_straight_5',
            'play_straight_6',
            'play_straight_7',
            'play_straight_8',
            'play_straight_9',
            'play_straight_10',
            'play_straight_11',
            'play_straight_12',
            'play_straight_13',
            'pass_turn'
        ]
    def reset(self):
        deck = np.arange(self.num_cards)
        np.random.shuffle(deck)
        self.player_hands = [np.zeros(self.num_cards, dtype=np.int8) for _ in range(self.num_players)]
        for i in range(self.num_players):
            self.player_hands[i][deck[i * self.num_cards_per_player:(i + 1) * self.num_cards_per_player]] = 1
        
        self.played_cards = np.zeros(self.num_cards, dtype=np.int8)
        
        self.current_player = 0
        self.rewards = [0 for _ in range(self.num_players)]
        self.current_played_cards =  [0 for _ in range(self.num_cards)]
        
        observation = self._get_obs()
        info = self._get_info()
        
        return observation,info
    def _get_observation(self):
        return {
            'player_hand': self.player_hands[0],     ## set train object is first player
            'opponent_hands': np.array([self.player_hands[i] for i in range(1,self.num_players)]),
            'played_cards': self.played_cards
        }
    def step(self, action):
        action_type = self.actions[action]
        curr_p = self.current_player
        
        reward = 0
        done = False
        
        if action_type == 'play_single':                                #Đánh 1 cây
            if np.sum(self.current_played_cards) == 1:
                idx_card_played = np.argmax(self.current_played_cards)
                if idx_card_played < 48:                    
                    idx_card_to_play = np.argmax(self.player_hands[curr_p][idx_card_played+1:] == 1) + idx_card_played + 1
                                                        # Đánh cây nhỏ nhất có thể
                    self.player_hands[idx_card_to_play] = 0
                    self.current_played_cards[idx_card_played] = 0
                    self.current_played_cards[idx_card_to_play] = 1
                    reward = 1
            
        elif action_type == 'play_two':                                  #Đánh 2
            if np.sum(self.current_played_cards) == 1:
                idx_card_played = np.argmax(self.current_played_cards)
                if idx_card_played < 51:
                    idx_card_to_play = np.argmax(self.player_hands[48:] == 1) 
                                                        # Đánh 2 nhỏ nhất có thể
                        
                    self.player_hands[idx_card_to_play] = 0
                    self.current_played_cards[idx_card_played] = 0
                    self.current_played_cards[idx_card_to_play] = 1
                    reward = 1
            
        elif action_type == 'play_pair':
            if np.sum(self.current_played_cards) == 2:
                idx_card_played = np.argmax(self.current_played_cards)
                if idx_card_played % 4 == 0:               # Đánh đôi đen
                    for i in range(idx_card_played+4,52,4):
                        if self.player_hands[i:i+2] == [1,1]:
                            self.player_hands[i:i+2] = [0,0]
                            
                            
                for i in range(idx_card_played,52,2):
                    if self.player_hands[i:i+2] == [1,1]:
                        reward = 2
            
        elif action_type == 'play_trio':
            reward = 3
            
        elif action_type == 'play_four_of_a_kind':
            reward = 4
            
        elif action_type == 'play_straight_3':
            reward = 3
            
        elif action_type == 'play_straight_4':
            reward = 4
            
        elif action_type == 'play_straight_5':
            reward = 5
            
        elif action_type == 'play_straight_6':
            reward = 6
            
        elif action_type == 'play_straight_7':
            reward = 7
            
        elif action_type == 'play_straight_8':
            reward = 8
            
        elif action_type == 'play_straight_9':
            reward = 9
            
        elif action_type == 'play_straight_10':
            reward = 10
            
        elif action_type == 'play_straight_11':
            reward = 11
            
        elif action_type == 'play_straight_12':
            reward = 12
            
        elif action_type == 'play_straight_13':
            reward = 13
            
        
        self.current_player = (self.current_player + 1) % self.num_players
        done = self.check_game_over()
        
        if done:
            reward = self._calculate_final_rewards()

        return self._get_observation(), reward, done, {}
        
        
    def check_game_over(self):
        return any(np.sum(hand) == 0 for hand in self.player_hands)

    def _calculate_final_rewards(self):
        rewards = [-10 * np.sum(hand) for hand in self.player_hands]
        rewards[self.current_player] += 100 
        return rewards[self.current_player]

            

In [54]:
cards_data

,0
0,3_bich
1,3_nhep
2,3_do
3,3_co
4,4_bich
5,4_nhep
6,4_do
7,4_co
8,5_bich
9,5_nhep


In [307]:
class player():
    def __init__(self):
        self.idx = None
        self.cards = np.zeros(52)
        self.valid_actions = np.zeros(17,dtype=np.int8)
        self.reward = 0
        self.num_hand_cards = np.sum(self.cards)
        self.current_played_cards = np.zeros(52)
        self.is_skip = False
        
    def get_played_cards_state(self):
        idx_first_card_played = np.argmax(self.current_played_cards)
        color = 0 if idx_first_card_played%4 <=2 else 1
        material = idx_first_card_played % 4
        num_played = np.sum(self.current_played_cards)
        
        # Kiểm tra dây
        indices = np.where(self.current_played_cards == 1)[0]
        all_congruent = np.all(indices % indices[0] == 0) if indices.size > 0 else False
        
        if num_played == 3:
            played_trio = not all_congruent
            played_four = False
        elif num_played == 4:
            played_trio = False
            played_four = not all_congruent
        else:
            played_trio, played_four = False,False
            
        return idx_first_card_played, color, material, num_played, played_trio, played_four
    
    #__________________________________________Lấy đôi____________________________________________#
    
    def get_pair_on_hand(self):
        idx_pair = []
        idx_color = []
        for i in range(0,52,2):
            if (cards[i],cards[i+1]) == (1,1):
                idx_pair = np.append(idx_pair,i)
                if i%4==0:
                    idx_color = np.append(idx_color,0)
                else:
                    idx_color = np.append(idx_color,1)
        return np.array(idx_pair),np.array(idx_color)
    #__________________________________________Lấy tam____________________________________________#
    def get_trio_on_hand(self):
        idx_trio = []
        material_trio = []

        for i in range(0, 52, 4):
            if sum([cards[i], cards[i+1], cards[i+2], cards[i+3]]) == 4:
                first_trio = (i, i+1, i+2)
                second_trio = (i, i+1, i+3)
                third_trio = (i+1, i+2, i+3)
                fourth_trio = (i, i+2, i+3)

                first_material = (0, 1, 2)
                second_material = (0, 1, 3)
                third_material = (1, 2, 3)
                fourth_material = (0, 2, 3)

                idx_trio.extend([first_trio, second_trio, third_trio, fourth_trio])
                material_trio.extend([first_material, second_material, third_material, fourth_material])

            elif [cards[i], cards[i+1], cards[i+2]] == [1, 1, 1]:
                idx_trio.append((i, i+1, i+2))
                material_trio.append((0, 1, 2))

            elif [cards[i], cards[i+1], cards[i+3]] == [1, 1, 1]:
                idx_trio.append((i, i+1, i+3))
                material_trio.append((0, 1, 3))

            elif [cards[i+1], cards[i+2], cards[i+3]] == [1, 1, 1]:
                idx_trio.append((i+1, i+2, i+3))
                material_trio.append((1, 2, 3))

            elif [cards[i], cards[i+2], cards[i+3]] == [1, 1, 1]:
                idx_trio.append((i, i+2, i+3))
                material_trio.append((0, 2, 3))  

            i += 3
        return np.array(idx_trio),np.array(material_trio)
    
    #__________________________________________Lấy tứ quý____________________________________________#
    
    def get_four_of_a_kind_on_hand(self):
        idx_four = []
        for i in range(0,52,4):
            if np.sum([self.cards[i], self.cards[i+1],self.cards[i+2],self.cards[i+3]]) == 4:
                idx_four = np.append(idx_four, i)
                i += 3
        return np.array(idx_four)
        
    #__________________________________________Lấy dây____________________________________________#
        
    def get_straight_on_hand(self,length):
        idx_straight = []
        for i in range(52-length*5):
            if self.cards[i] == 1:
                straight = [i]
                idx_card_into_straight = i
                for _ in range(length-1):
                    idx_card_into_straight += 4
                    if self.cards[idx_card_into_straight] == 1:
                        straight = np.append(straight, idx_card_into_straight)

                if len(straight) == length:
                    idx_straight = np.append(idx_straight,straight[0])
        return np.array(idx_straight)     
    
    #__________________________________________Lấy actions____________________________________________#
    
    def get_valid_action(self):
        if self.is_skip:
            return self.valid_actions
        idx_first_card_played, color_played, material_played, num_played,played_trio,played_four = self.get_played_cards_state()
       
        if num_played == 1:
            biggest_card = np.where(self.cards == 1)[0][-1]
            if idx_first_card_played < biggest_card:
                self.valid_actions[0] = 1
            if (biggest_card > 47) and (biggest_card > idx_first_card_played):
                self.valid_actions[1] = 1
                
        elif num_played == 2:
            pair_to_play,color_to_play = self.get_pair_on_hand()
            if pair_to_play.size > 0:
                for idx_first_card_in_pair in pair_to_play:
                    if (idx_first_card_in_pair > idx_first_card_played) and (idx_first_card_in_pair%2 == color_played):
                        self.valid_actions[2] = 1
                        
        elif (num_played == 3) and played_trio:
            if (self.get_trio_on_hand()[0].size > 0) :
                self.valid_actions[3] = 1
        elif num_played == 3:
            if self.get_straight_on_hand(3).size > 0:
                self.valid_actions[5] = 1
                
        elif num_played == 4 and played_four:
            if self.get_four_of_a_kind_on_hand().size > 0:
                self.valid_actions[4] = 1
        elif num_played == 4: 
            if self.get_straight_on_hand(4).size > 0:
                self.valid_actions[6] = 1  
                
        elif num_played == 5:
            if self.get_straight_on_hand(5).size > 0:
                self.valid_actions[7] = 1  
                
        elif num_played == 6:
            if self.get_straight_on_hand(6).size > 0:
                self.valid_actions[8] = 1  
                
        elif num_played == 7:
            if self.get_straight_on_hand(7).size > 0:
                self.valid_actions[9] = 1 
                
        elif num_played == 8:
            if self.get_straight_on_hand(8).size > 0:
                self.valid_actions[10] = 1   
                
        elif num_played == 9:
            if self.get_straight_on_hand(9).size > 0:
                self.valid_actions[11] = 1  
                
        elif num_played == 10:
            if self.get_straight_on_hand(10).size > 0:
                self.valid_actions[12] = 1  
                
        elif num_played == 11:
            if self.get_straight_on_hand(11).size > 0:
                self.valid_actions[13] = 1  
                
        elif num_played == 12:
            if self.get_straight_on_hand(12).size > 0:
                self.valid_actions[14] = 1 
                
        elif num_played == 13:
            if self.get_straight_on_hand(13).size > 0:
                self.valid_actions[15] = 1
                
        self.valid_actions[16] =1
        return self.valid_actions

            
            

In [214]:
action_data

,0
0,play_single
1,play_two
2,play_pair
3,play_trio
4,play_four_of_a_kind
5,play_straight_3
6,play_straight_4
7,play_straight_5
8,play_straight_6
9,play_straight_7


In [215]:
cards = np.random.randint(2, size=52)
cards_data[0][cards == 1].values

array(['3_bich', '3_do', '3_co', '4_bich', '4_nhep', '4_do', '4_co',
       '7_bich', '7_do', '8_bich', '8_nhep', '8_co', '9_bich', '9_nhep',
       '9_co', '10_nhep', '10_do', 'j_co', 'q_bich', 'k_do', 'k_co',
       'A_do', '2_bich'], dtype=object)

In [262]:
cards = np.ones(52,dtype=np.int8)

In [305]:
current_played_cards = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=np.int8)
current_played_cards[[4,8,12,16,18]] = 1

In [308]:
player1 = player()
player1.cards = cards
player1.current_played_cards = current_played_cards
player1.valid_action = player1.get_valid_action()
player1.valid_action 

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int8)

In [232]:
player1.get_pair_on_hand()

array([ 2.,  4.,  6., 20., 24., 42.])

In [268]:
action_data

,0
0,play_single
1,play_two
2,play_pair
3,play_trio
4,play_four_of_a_kind
5,play_straight_3
6,play_straight_4
7,play_straight_5
8,play_straight_6
9,play_straight_7


# Thêm check bài đánh ra phải lớn hơn bài đã đánh 